# Распознавание объектов на изображениях 

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.


Грузим библиотеки :
точно нужны слои,  optimizer , cifar10

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D


from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline 

from tensorflow.keras import utils
# Количество классов изображений
nb_classes = 10
# Названия классов из набора данных CIFAR-10
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train10 = utils.to_categorical(y_train, nb_classes)

y_test10 = utils.to_categorical(y_test, nb_classes)

#x_train=np.broadcast_to(x_train[..., None],(60000,28,28,3))
#x_test=np.broadcast_to(x_test[..., None],(10000,28,28,3))

In [ ]:
from tensorflow.keras.layers import Add , Input,  Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
import tensorflow
from tensorflow.keras.utils import plot_model,to_categorical



задаем размер входа

In [ ]:
# Размер изображений  CIFAR10
img_width, img_height, chanel_n = 32, 32, 3
#img_width, img_height, chanel_n = 28, 28, 3

print(x_train.shape)
#пакет для генератора
batch_size = 128


#Свой классификатор 

In [ ]:
from tensorflow.keras.layers import BatchNormalization

layer_size = 10
models = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
models.add(Conv2D(128,(3,3), activation='sigmoid',input_shape = (32,32,3))) # только свертка
models.add(BatchNormalization())
for i in range(layer_size):
  models.add(Conv2D(128,(3,3), activation='sigmoid')) # только свертка
  models.add(BatchNormalization())

models.add(Flatten()) # векторим вход
models.add(Dense(100,activation='sigmoid'))
models.add(Dense(10,activation='softmax'))


models.summary()


print(models.layers[-1].input_shape)

print(models.layers[-1].output_shape)


Посмотрим, что вышло

In [ ]:
# компилияция модели
models.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ["accuracy"])

In [ ]:
hist = models.fit(x = x_train,y = y_train10, epochs = 1, batch_size = 128, validation_data = (x_test, y_test10), verbose = 1)

test_score = models.evaluate(x_test, y_test10)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

In [ ]:

predict1=models.predict(x_train[:20])
print(predict1.shape)

Посмотрим , что распознается

In [ ]:
n=1

plt.imshow(x_train[n][:,:,:])
plt.title(classes[np.argmax(predict1[n,:])])
plt.show()

In [ ]:
predict1_test=models.predict(x_test[:20])


Сделаем немного тестов

In [ ]:
n=5

plt.imshow(x_test[n][:,:,:])
plt.title(classes[np.argmax(predict1_test[n,:])])
plt.show()

In [ ]:

# Создадим экземпляр оптимизатора. 
optimizer = tensorflow.keras.optimizers.SGD(learning_rate = 1e-3) 
# Instantiate a loss function. 
loss_fn = tensorflow.keras.losses.MeanSquaredError() #берем, т.к. виднее изменения(from_logits=True) 
# Подготовим тренировочный датасет. 
batch_size = 64 
epochs=2  # учим немного, т.к. задача посмотреть , что происходит с градиентом
train_dataset = tensorflow.data.Dataset.from_tensor_slices((x_train[:1000,:,:,:], to_categorical(y_train[:1000,0]))) 
train_dataset = train_dataset.shuffle(buffer_size = 1024).batch(batch_size) 

def my_train(model=None, epochs=None,batch_size=64,train_dataset=train_dataset ):
  # Итерируем по эпохам.
  grad_log=[]
  w_log=[]


  for epoch in range(epochs): 
    print('Начинаем эпоху %d' % (epoch,)) 
    # Итерируем по пакетам в датасете. 
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset): 
      # Откроем GradientTape чтобы записать операции # выполняемые во время прямого прохода, 
      #включающего автодифференцирование. 
      with tensorflow.GradientTape() as tape: 
        # Запустим прямой проход слоя. 
        # Операции применяемые слоем к своим 
        # входным данным будут записаны 
        # на GradientTape. 
        logits = model(x_batch_train) 
        # Logits для пакета  - выходы модели 
        # Вычислим значение потерь для этого пакета. 
        y_s=y_batch_train.numpy().shape
        y  = tensorflow.reshape( y_batch_train,shape = (y_s[0],10,1))
        l = tensorflow.reshape( logits,shape = (y_s[0],10,1))
        # вызываем лосс
        loss_value = loss_fn(y, l) 
        # Используем gradient tape для автоматического извлечения градиентов 
        # обучаемых переменных относительно потерь. 
        grads = tape.gradient(loss_value, model.trainable_weights) 
        g_g = []
        w_w = []
        # пишем логи для сохранения значений градиента по одному ядру из слоя (осредняем по абсолютным значениям) и 
        # веса по одной цепи (0-й канал распространения активности)
        for g_s in grads:
          if len(g_s.numpy().shape) == 1:
         
            g_g.append(g_s.numpy()[0])
          if len(g_s.numpy().shape) == 2:
          
            g_g.append(np.mean(np.abs((g_s.numpy()[:,0])))) 
          if len(g_s.numpy().shape) == 3:
          
            g_g.append(np.mean(np.abs(g_s.numpy()[:,:,0]))) 
          if len(g_s.numpy().shape) == 4:
          
            g_g.append(np.mean(np.abs(g_s.numpy()[:,:,0,0])) ) 

        for w_s in model.trainable_weights:
          if len(w_s.numpy().shape) == 1:
            w_w.append(w_s.numpy()[0])
          if len(w_s.numpy().shape) == 2:
            w_w.append(w_s.numpy()[0,0])
          
          if len(w_s.numpy().shape) == 3:
            w_w.append(w_s.numpy()[0,0,0])
         
          if len(w_s.numpy().shape) == 4:
            w_w.append(w_s.numpy()[0,0,0,0])
              
        # добавляем текущие логи по слоям к общей записи
        grad_log.append(g_g)
        w_log.append(w_w)
        # Выполним один шаг градиентного спуска обновив # значение переменных минимизирующих потери. 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        # Пишем лог каждые 1000 пакетов. 
        if step % 1000 == 0: 
          print('Потери на обучении (для одного пакета) на шаге %s: %s' % (step, float(loss_value))) 
          print('Уже увидели: %s примеров' % ((step + 1) * batch_size))
  grad_log=np.array(grad_log)
  w_log=np.array(w_log)
  return grad_log, w_log, model, x_batch_train

Вывод значений градиента по слоям

In [ ]:
def visual_grad(grad_log = None, numb_layer = -1):
  s_g = grad_log.shape
  grad_log = grad_log/np.max(grad_log,axis = 1).reshape((s_g[0],1))
  #print(grad_log[0,:])
  plt.figure(figsize = (16,5))
  plt.title('Градиент по слоям')
  plt.xlabel('№ layer')
  plt.ylabel('grad')
  plt.grid()
  plt.plot(np.abs(grad_log[0,:numb_layer]),label='step 0')
  plt.plot(np.abs(grad_log[s_g[0] // 10,:numb_layer]),label='step '+str(s_g[0] // 10))
  plt.plot(np.abs(grad_log[s_g[0] // 3,:numb_layer]),label='step '+str(s_g[0] // 3))
  plt.plot(np.abs(grad_log[s_g[0]-1 ,:numb_layer]),label='step  '+str(s_g[0] ))
  plt.legend()
  plt.show()

In [ ]:
grad_log,w_log,models, x_batch_train = my_train( model = models, epochs = 1,batch_size = 10 , train_dataset = train_dataset)

In [ ]:
visual_grad(grad_log = grad_log, numb_layer = grad_log.shape[1])

ПОстроим сложный блок (Rez-Inception)

In [ ]:
inputs = Input(shape=(32,32,3), name='CIFAR10rez') 

x = Conv2D(64,3, activation = 'relu', name = 'conv_1')(inputs)
x = BatchNormalization()(x)
x = Conv2D(64,3, activation = 'relu', name = 'conv_2')(x) 
# можно добавлять блоки сверток, но следует следить за размером
# padding ='valid' - размер выходов постепенно уменьшается 
#ResNet - блок
x_shortcut = x # +x
x = Conv2D(64,3, activation = 'relu', padding = 'same',name = 'conv_31')(x) # H
x = Conv2D(64,3, activation = 'relu', padding = 'same',name = 'conv_41')(x) 
x = Add()([x, x_shortcut]) # F = H +x
x = BatchNormalization()(x)
# нелинейная трансформация и уменьшение размера в 2 раза
x = MaxPool2D()(x) 
x_shortcut = x
x = Conv2D(64,3, activation = 'relu', padding = 'same', name = 'conv_32')(x) 
x = Conv2D(64,3, activation = 'relu', padding = 'same', name = 'conv_42')(x)
x = Add()([x, x_shortcut])
x = BatchNormalization()(x)
x_shortcut = x

x = Conv2D(64,3, activation = 'relu', padding = 'same', name = 'conv_3')(x) 
x = Conv2D(64,3, activation = 'relu',  padding = 'same', name = 'conv_4')(x)
x = Add()([x, x_shortcut])  
x = MaxPool2D((2,2),(2,2))(x)
x = Flatten()(x)
x = Dense(64, activation = 'relu', name = 'dense_1')(x) 
 

outputs = Dense(10,activation = 'sigmoid' ,name = 'predictions')(x) 
model1 = Model(inputs = inputs, outputs = outputs) 

In [ ]:
plot_model(model1,to_file = 'model.png')

In [ ]:
grad_log,w_log,model1, x_batch_train = my_train( model = model1, epochs = 1,batch_size = 1000 , train_dataset = train_dataset)

In [ ]:
visual_grad(grad_log = grad_log, numb_layer = grad_log.shape[1])

In [ ]:
model1.summary()

In [ ]:
from tensorflow.keras.layers import concatenate 


inputs = Input(shape = (32,32,3), name = 'cifar_inception') 

x = Conv2D(64,3, activation='relu', name='conv_1')(inputs)
x = BatchNormalization()(x)
x = Conv2D(64,3, activation='relu', name='conv_2')(x) 
# можно добавлять блоки сверток, но следует следить за размером
# padding ='valid' - размер выходов постепенно уменьшается 
#Inception - блок
x_shortcut = x
x0 = Conv2D(64,1, activation='relu', padding='same',name='conv_131')(x) 
x1 = Conv2D(64,1, activation='relu', padding='same',name='conv_231')(x) 
x1 = Conv2D(64,5, activation='relu', padding='same',name='conv_341')(x1) 
x2 = Conv2D(64,1, activation='relu', padding='same',name='conv_431')(x) 
x2 = Conv2D(64,3, activation='relu', padding='same',name='conv_541')(x2) 
x3 = Conv2D(64,3, activation='relu', padding='same',name='conv_631')(x) 
x3 = Conv2D(64,1, activation='relu', padding='same',name='conv_741')(x3) 
x = concatenate([x0,x1,x2,x3],axis=-1)
x = BatchNormalization()(x)
x = Conv2D(64,3, activation='relu', padding='same', name='conv_ee32')(x) 
x = Add()([x, x_shortcut])
# нелинейная трансформация и уменьшение размера в 2 раза
x = MaxPool2D()(x) 
x_shortcut = x
x = Conv2D(64,3, activation='relu', padding='same', name='conv_32')(x) 
x = Conv2D(64,3, activation='relu', padding='same', name='conv_42')(x)
x = Add()([x, x_shortcut])
x = BatchNormalization()(x)
x_shortcut = x
x = Conv2D(64,3, activation = 'relu', padding = 'same',name = 'conv_33')(x) 
x = Conv2D(64,3, activation = 'relu', padding = 'same',name = 'conv_43')(x)  
x = Add()([x, x_shortcut])  
x = BatchNormalization()(x)
x_shortcut = x
x = Conv2D(64,3, activation = 'relu', padding = 'same', name = 'conv_3')(x) 
x = Conv2D(64,3, activation = 'relu',  padding = 'same', name = 'conv_4')(x)
x = Add()([x, x_shortcut])  
x = MaxPool2D((2,2),(2,2))(x)
x = Flatten()(x)
x = Dense(64, activation = 'relu', name = 'dense_1')(x) 
 

outputs = Dense(10,activation = 'sigmoid' ,name = 'predictions')(x) 
model1 = Model(inputs = inputs, outputs = outputs) 

grad_log,w_log,model1, x_batch_train = my_train( model = model1, epochs = 1,batch_size = batch_size , train_dataset = train_dataset)


In [ ]:
plot_model(model1,to_file='model.png')

In [ ]:
visual_grad(grad_log=grad_log, numb_layer=grad_log.shape[1])

In [ ]:
model1.summary()